```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de datos no estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2024
Profesor: Facundo Santiago, Javier Ignacio Garcia Fronti
```

# ¿Qué tan memorable es un video?: Solución base utilizando videos, images y audio

Este notebook les permitirá tener a disposición todos los recursos que se vieron en durante la materia. Todos los fragmentos de código se descargaran utilizando la sección preparación del ambiente.

## Preparación del ambiente

### Sets de datos

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/ground_truth.csv --directory-prefix ./Data/ --quiet

### Librerías de codigo que se utilizan durante el curso

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/solucion_base.txt --quiet
!pip install -r solucion_base.txt --quiet

Descargamos todo el código con utilidades que se vieron en el curso desde el repositorio de la materia:

In [ ]:
!git clone -n --depth=1  https://github.com/santiagxf/M72109
!cd M72109 && git sparse-checkout set --no-cone m72109 && git checkout
!mv M72109/m72109 . && rm -rf M72109

Descargamos la etiquetas de YamNET en el caso de utilizarlas:

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/Models/yamnet/yamnet_class_map.csv \
    --directory-prefix ./Models/yamnet/ --quiet --no-clobber

Descargamos modelos de Word2Vec en ingles:

In [ ]:
!wget https://santiagxf.blob.core.windows.net/public/Word2Vec/model-en.bin --directory-prefix ./Models/Word2Vec --quiet

### Imports

In [ ]:
import pandas as pd
import numpy as np
import librosa
import os
import soundfile as sf
import torch
import datasets
import transformers
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, SpatialDropout1D
import tensorflow_datasets as tfds

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from m72109.audio.plotting import plot_audio_embeddings
from m72109.nlp.normalization import TextNormalizer, TweetTextNormalizer
from m72109.nlp.transformation import Word2VecVectorizer, PadSequenceTransformer, split_text_with_context

import azure.cognitiveservices.speech as speechsdk
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

## Solución

Cuentan con 2 tipos de anotaciones para cada uno de los fragmentos de video disponibles:
 - **memorability_score:** Representa el puntaje de memorabilidad de la secuencia en particular, desde 0 a 1. Valores más grandes son mejores.
 - **memorable:** Variable categórica que representa si un video es memorable o no. Un video con `memorability_score` superior a `0.5` es marcado como memorable (`1`), sino es marcado como no memorable (`0`)

In [ ]:
labels = pd.read_csv('Data/ground_truth.csv')

In [ ]:
labels.head(2)

In [ ]:
labels[labels['movie_name'] == "127 hours"]['movie_name'][0]

> Note que aquí nuestras muestras son "secuencias" de determinadas películas. En total dispone de 609 secuencias con el nombre `sequence_name`. El mismo nombre se generó automáticamente concatenando el nombre de la pelicula a la que pertenece la secuencia (movie_name), seguido del segundo en el que comienza la secuencia, seguido del segundo en el que termina, seguido de un numero que indica el número de secuencia. Por ejemplo, la secuencia `127_hours_2000_2010_1` es un fragmento de la pelicula "127 hours", que va desde el segundo 2000 (00:33:20 hrs.) al segundo 2010 (00:33:30 hrs.) y es el fragmento número 1. Esta información es totalmente irrelevante para el problema de memorabilidad.

### Datos no estructurados

Dado que los conjuntos de datos pueden ser de gran tamaño, descargue solo los datos que necesita:

#### Para utilizar los cuadros (frames)

In [ ]:
!wget -N https://santiagxf.blob.core.windows.net/public/Memorability/frames.tar.gz --directory-prefix ./Data/Raw/
!tar zxvf ./Data/Raw/frames.tar.gz --directory ./Data/Raw/

> En el directorio `Data/Raw/frames` encontraran todos los cuadros extraidos de cada una de las secuencias. Los cuadros están disponibles cada 2 segundos por lo cual disponen de 5 cuadros por cada secuencia. Todos los cuadros pertenecientes a una secuencia están almacenados en una carpeta con el mismo nombre de la secuencia a la que pertenece, por lo que la carpeta `127_hours_2000_2010_1` contiene todos los cuadros pertenecientes a la secuencia `127_hours_2000_2010_1` en el conjunto de datos de anotaciones.

##### Tip: Leyendo el dataset

En este ejemplo, las etiquetas no están en el conjunto de datos, sino que se encuentran en un archivo estructurado. Necesitaremos utilizar el directorio donde se encuentra la imagen para ubicar el label correspondiente.

Para ello, primero leeremos todas las imagenes disponibles y ubicaremos cual es su etiqueta correspondiente (memorable o no memorable). Todas las imágenes del mismo directorio tendran la misma etiqueta. El siguiente código realiza lo mencionado:

In [ ]:
memorable_labels = []
for root, _, filepaths in os.walk('Data/Raw/frames'):
  for filepath in filepaths:
    sequence_name = os.path.basename(root)
    if sequence_name in np.unique(labels['sequence_name'].values):
      memorable_labels.append(labels[labels['sequence_name'] == sequence_name]['memorabable'].iloc[0])
    else:
       memorable_labels.append(0)


Para poder crear un conjunto de datos combinando las imágenes con sus etiquetas deberemos definir una función para leer las imagenes dado un directorio:

In [ ]:
import tensorflow as tf
from tensorflow.data.experimental import AUTOTUNE

def parse_image(filename, label, channels:int=3, img_size:int=224):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=channels)
    image_resized = tf.image.resize(image_decoded, [img_size, img_size])

    return image_resized, label

Finalmente, construiremos una función que genera el conjunto de datos que necesitamos:

Note que:

1. `tf.data.Dataset.list_files` lista todos los archivos de un directorio dado. Estos seran los predictores.
1. `tf.data.Dataset.from_tensor_slices` construye un dataset de TensorFlow a partir de un arreglo de `Numpy` o `Pandas`. Estos serán las etiquetas.
1. `tf.data.Dataset.zip` construye un dataset combinando 2 datasets previamente creados. En este caso lo utilizaremos para unir los predictores (creados en el paso 1) y las etiquetas (creadas en el paso 2).

In [ ]:
def create_dataset(filenames, labels, batch_size=64, is_training=True):
    X_train = tf.data.Dataset.list_files(filenames, shuffle=False)
    y_train = tf.data.Dataset.from_tensor_slices(np.array(labels))
    dataset = tf.data.Dataset.zip((X_train, y_train))
    dataset = dataset.map(parse_image, num_parallel_calls=AUTOTUNE)

    if is_training:
        dataset = dataset.shuffle(buffer_size=500)

    # dataset = dataset.batch(batch_size) # Si necesita implementar batching
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)

    return dataset

Construimos el dataset:

In [ ]:
train_ds = create_dataset(filenames = 'Data/Raw/frames/*/*.jpg', labels = memorable_labels, is_training=True)

Verifique los datos que obtendra:

In [ ]:
for feat, label in train_ds.take(1):
  print('Features: shape', feat.shape)
  print('Label: sample value', label.numpy())

In [ ]:
# completar solución

#### Para utilizar los audios

In [ ]:
!wget -N https://santiagxf.blob.core.windows.net/public/Memorability/audios.tar.gz --directory-prefix ./Data/Raw/
!tar zxvf ./Data/Raw/audios.tar.gz --directory ./Data/Raw/

> En el directorio `Data/Raw/audios` encontraran todos los audios correspondientes a cada una de las secuencias de los videos. Cada audio tiene el mismo nombre de la secuencia a la que pertenece, por lo que el archivo `127_hours_2000_2010_1.wav` pertenece a la secuencia `127_hours_2000_2010_1` en el conjunto de datos de anotaciones. Los audios están en formato `wav`, que si bien ocupan mayor espacio, son más sencillos de procesar.

In [ ]:
# completar solución